In [2]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 

In [3]:
train = pd.read_csv("modifiedabalone.csv", header=None)
train.head()

,0,1,2,3,4,5,6,7,8
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,1
1,1,0.605,0.475,0.180,0.9365,0.3940,0.2190,0.295,1
2,2,0.680,0.560,0.165,1.6390,0.6055,0.2805,0.460,1
3,2,0.600,0.475,0.150,1.0075,0.4425,0.2210,0.280,1
4,1,0.565,0.425,0.135,0.8115,0.3410,0.1675,0.255,1


In [12]:
features = train.columns[0:8]
X = train[features]
y = train[8]
#X.head()
X

,0,1,2,3,4,5,6,7
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500
1,1,0.605,0.475,0.180,0.9365,0.3940,0.2190,0.2950
2,2,0.680,0.560,0.165,1.6390,0.6055,0.2805,0.4600
3,2,0.600,0.475,0.150,1.0075,0.4425,0.2210,0.2800
4,1,0.565,0.425,0.135,0.8115,0.3410,0.1675,0.2550
...,...,...,...,...,...,...,...,...
4172,2,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490
4173,1,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605
4174,1,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080
4175,2,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [6]:
print(X_train.shape,X_test.shape)

(3341, 8) (836, 8)


In [9]:
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)
print(X_train[75:130])

[[3.     0.57   0.42   0.13   0.7745 0.3535 0.1505 0.2365]
 [1.     0.495  0.4    0.12   0.6605 0.2605 0.161  0.19  ]
 [3.     0.52   0.4    0.13   0.5825 0.233  0.1365 0.18  ]
 [2.     0.485  0.365  0.12   0.5885 0.27   0.131  0.175 ]
 [2.     0.38   0.32   0.115  0.6475 0.323  0.1325 0.164 ]
 [2.     0.7    0.525  0.19   1.6015 0.707  0.365  0.43  ]
 [3.     0.525  0.385  0.13   0.607  0.2355 0.125  0.195 ]
 [1.     0.495  0.385  0.135  0.709  0.211  0.1375 0.262 ]
 [2.     0.635  0.475  0.15   1.1845 0.533  0.307  0.291 ]
 [2.     0.55   0.405  0.125  0.651  0.2965 0.137  0.2   ]
 [2.     0.625  0.5    0.16   1.217  0.5725 0.207  0.355 ]
 [1.     0.615  0.5    0.17   1.054  0.4845 0.228  0.295 ]
 [3.     0.535  0.4    0.135  0.775  0.368  0.208  0.2055]
 [3.     0.175  0.125  0.05   0.0235 0.008  0.0035 0.008 ]
 [1.     0.595  0.485  0.15   1.0835 0.5305 0.231  0.276 ]
 [1.     0.62   0.495  0.175  1.806  0.643  0.3285 0.725 ]
 [3.     0.475  0.38   0.12   0.441  0.1785 0.0885 0.150

In [7]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=10.0):
   # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

def gm(y_predict,y_test):
    test_min=0
    test_max=0
    pred_min=0
    pred_max=0
    y_test=np.asarray(y_test)
    for i in range(0,836):
        if(y_test[i]==1):
             test_min=test_min+1
        else:
             test_max=test_max+1
    print("y_test min",test_min)       
    print("y_test max",test_max)
    for i in range(0,836):
        if(y_predict[i]==1 and y_predict[i]==y_test[i]):
             pred_min=pred_min+1
        elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
             pred_max=pred_max+1
    print("y_pred min",pred_min)       
    print("y_pred max",pred_max)
    se=pred_min/test_min
    sp=pred_max/test_max
    print(se,sp)
    gm=math.sqrt(se*sp)
    print("GM",gm)

# FSVM using Hyperplane

In [8]:
from cvxopt import matrix
class HYP_SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def m_func(self, X_train,X_test, y):
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features= X_test.shape
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j])
               # print(K[i,j])
        X_train=np.asarray(X_train)
        X_test=np.asarray(X_test)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j])
               # print(K[i,j])
        print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)
        typ=1
        if(typ==1):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.2
        if(typ==2):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=103/4074
        r_min=1
        self.m=func[0:103]*r_min
        print(self.m.shape)
        self.m=np.append(self.m,func[103:4177]*r_max)
        print(self.m.shape)
        
 ##############################################################################


    def fit(self, X_train,X_test, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features = X_test.shape
        # Gram matrix

        print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None        
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [9]:

if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        
        clf = HYP_SVM(C=100.0)
        typ=2
        clf.m_func(X_train,X_test,y_train)
        clf.fit(X_train,X_test, y_train)
        y_predict = clf.predict(X_test)
        gm(y_predict,y_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print("Accuracy",correct/len(y_predict))

    hyp_svm()    

(3341, 3341)
     pcost       dcost       gap    pres   dres
 0: -1.4904e+04 -2.0036e+06  2e+06  2e-02  8e-13
 1: -1.3268e+04 -1.4210e+05  1e+05  4e-04  8e-13
 2: -1.5305e+04 -5.5521e+04  4e+04  3e-05  8e-13
 3: -1.5484e+04 -5.3758e+04  4e+04  3e-05  8e-13
 4: -1.6200e+04 -3.0338e+04  1e+04  5e-06  8e-13
 5: -1.6293e+04 -2.9163e+04  1e+04  3e-06  7e-13
 6: -1.6519e+04 -2.4440e+04  8e+03  1e-06  8e-13
 7: -1.6658e+04 -2.0918e+04  4e+03  6e-07  7e-13
 8: -1.6753e+04 -1.8317e+04  2e+03  2e-07  7e-13
 9: -1.6785e+04 -1.7377e+04  6e+02  5e-08  7e-13
10: -1.6797e+04 -1.6906e+04  1e+02  5e-09  8e-13
11: -1.6799e+04 -1.6829e+04  3e+01  8e-11  8e-13
12: -1.6799e+04 -1.6820e+04  2e+01  4e-11  7e-13
13: -1.6800e+04 -1.6806e+04  6e+00  9e-12  8e-13
14: -1.6800e+04 -1.6805e+04  6e+00  7e-12  8e-13
15: -1.6800e+04 -1.6803e+04  4e+00  5e-13  9e-13
16: -1.6800e+04 -1.6802e+04  2e+00  2e-13  8e-13
17: -1.6800e+04 -1.6801e+04  1e+00  3e-13  7e-13
18: -1.6800e+04 -1.6801e+04  1e+00  2e-13  7e-13
19: -1.6

In [10]:
clf_svm = svm.SVC(kernel='rbf', gamma=0.001, C=100)
clf_svm.fit(X_train, y_train)
y_pred_svm = clf_svm.predict(X_test) 
acc_svm = accuracy_score(y_test, y_pred_svm)
print ("Overall RBF KERNEL SVM accuracy: ",acc_svm)

Overall RBF KERNEL SVM accuracy:  0.9772727272727273


# Normal SVM using CVXOPT

In [11]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = gaussian_kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [12]:

if __name__ == "__main__":
    import pylab as pl           
    def normal_svm():
        
        clf = SVM(C=100.0)
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        gm(y_predict,y_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print("Accuracy",correct/len(y_predict))

    normal_svm()    

(3341, 3341)
     pcost       dcost       gap    pres   dres
 0: -1.4904e+04 -2.0036e+06  2e+06  2e-02  8e-13
 1: -1.3268e+04 -1.4210e+05  1e+05  4e-04  8e-13
 2: -1.5305e+04 -5.5521e+04  4e+04  3e-05  8e-13
 3: -1.5484e+04 -5.3758e+04  4e+04  3e-05  8e-13
 4: -1.6200e+04 -3.0338e+04  1e+04  5e-06  8e-13
 5: -1.6293e+04 -2.9163e+04  1e+04  3e-06  7e-13
 6: -1.6519e+04 -2.4440e+04  8e+03  1e-06  8e-13
 7: -1.6658e+04 -2.0918e+04  4e+03  6e-07  7e-13
 8: -1.6753e+04 -1.8317e+04  2e+03  2e-07  7e-13
 9: -1.6785e+04 -1.7377e+04  6e+02  5e-08  7e-13
10: -1.6797e+04 -1.6906e+04  1e+02  5e-09  8e-13
11: -1.6799e+04 -1.6829e+04  3e+01  8e-11  8e-13
12: -1.6799e+04 -1.6820e+04  2e+01  4e-11  7e-13
13: -1.6800e+04 -1.6806e+04  6e+00  9e-12  8e-13
14: -1.6800e+04 -1.6805e+04  6e+00  7e-12  8e-13
15: -1.6800e+04 -1.6803e+04  4e+00  5e-13  9e-13
16: -1.6800e+04 -1.6802e+04  2e+00  2e-13  8e-13
17: -1.6800e+04 -1.6801e+04  1e+00  3e-13  7e-13
18: -1.6800e+04 -1.6801e+04  1e+00  2e-13  7e-13
19: -1.6